In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda"  # cuda:number or cpu
        self.data = "nci"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load nci


In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cuda"

for i, (train_index, test_index) in enumerate(kfold.split(np.arange(pos_num))):
    sampler = Sampler(res, train_index, test_index, null_mask, i)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.792296 auc:0.4945
epoch:  20 loss:0.578193 auc:0.7050
epoch:  40 loss:0.485097 auc:0.7516
epoch:  60 loss:0.396437 auc:0.7903
epoch:  80 loss:0.350800 auc:0.8099
epoch: 100 loss:0.328341 auc:0.8190
epoch: 120 loss:0.316256 auc:0.8238
epoch: 140 loss:0.308904 auc:0.8263
epoch: 160 loss:0.304076 auc:0.8280
epoch: 180 loss:0.300733 auc:0.8292
epoch: 200 loss:0.298338 auc:0.8303
epoch: 220 loss:0.296568 auc:0.8313
epoch: 240 loss:0.295231 auc:0.8321
epoch: 260 loss:0.294200 auc:0.8326
epoch: 280 loss:0.293386 auc:0.8329
epoch: 300 loss:0.292731 auc:0.8332
epoch: 320 loss:0.292198 auc:0.8333
epoch: 340 loss:0.291761 auc:0.8334
epoch: 360 loss:0.291401 auc:0.8334
epoch: 380 loss:0.291103 auc:0.8334
epoch: 400 loss:0.290854 auc:0.8335
epoch: 420 loss:0.290646 auc:0.8335
epoch: 440 loss:0.290470 auc:0.8335
epoch: 460 loss:0.290319 auc:0.8335
epoch: 480 loss:0.290189 auc:0.8335
epoch: 500 loss:0.290075 auc:0.8335
epoch: 520 loss:0.289979 auc:0.8336
epoch: 540 loss:0.289889 auc

In [5]:
true_datas.reset_index(drop=True).to_csv("true_nci.csv")
predict_datas.reset_index(drop=True).to_csv("pred_nci.csv")